In [138]:
import pymongo
from bson.objectid import ObjectId
import pprint
from datetime import datetime, timedelta

pp = pprint.PrettyPrinter(depth=4)

In [139]:
my_client = pymongo.MongoClient("mongodb://localhost:27017/")

In [140]:
my_db = my_client["shop"]

Вам необхідно змоделювати інтернет-магазин який торгує найрізноманітнішими товарами (тобто у яких різний набір властивостей) - items.

In [141]:
items_collection = my_db["items"]

1) Створіть декілька товарів з різним набором властивостей

In [142]:
items_insert = [
    {
        "category": "Phone",
        "model": "iPhone 14",
        "producer": "Apple",
        "price": 900,
        "display": 6.1
    },
    {
        "category": "Phone",
        "model": "iPhone 13",
        "producer": "Apple",
        "price": 800,
        "display": 6.1
    },
    {
        "category": "Phone",
        "model": "Galaxy s23",
        "producer": "Samsung",
        "price": 850,
        "display": 6.2
    },

    {
        "category": "Watch",
        "model": "Series 8",
        "producer": "Apple",
        "price": 400,
        "feature": "ECG",
        "material": "aluminium"
    },
    {
        "category": "Watch",
        "model": "Galaxy Watch5",
        "producer": "Samsung",
        "price": 350,
        "feature": "Sleep Coaching",
        "material": "aluminium"
    },

    {
        "category": "Headphones",
        "model": "AirPods 3",
        "producer": "Apple",
        "price": 180,
        "type": "wireless",
        "feature": "Force sensor",
        "connectivity": "Bluetooth 5.0"
    },
    {
        "category": "Headphones",
        "model": "Galaxy Buds2 Pro",
        "producer": "Samsung",
        "price": 125,
        "feature": "IPX7 water-resistant",
        "Connectivity": "Bluetooth 5.0"
    }
]

In [143]:
items_collection.insert_many(items_insert)

2) Напишіть запит, який виводіть усі товари (відображення у JSON)

In [185]:
for item in items_collection.find({}):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f1134f'),
 'category': 'Phone',
 'display': 6.1,
 'model': 'iPhone 14',
 'price': 1050,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11350'),
 'category': 'Phone',
 'display': 6.1,
 'model': 'iPhone 13',
 'price': 800,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11351'),
 'category': 'Phone',
 'display': 6.2,
 'model': 'Galaxy s23',
 'price': 850,
 'producer': 'Samsung'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11352'),
 'category': 'Watch',
 'discount': True,
 'feature': 'ECG',
 'material': 'aluminium',
 'model': 'Series 8',
 'price': 410,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'discount': True,
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 360,
 'producer': 'Samsung'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11354'),
 'category': 'Headphones',
 'connectivity': 'Bluetooth 5.0',
 'discount': True,
 'feature': 'Force sensor',
 'model

3) Підрахуйте скільки товарів у певної категорії

In [145]:
for item in items_collection.aggregate([
   {
       "$group" : { "_id" : '$category', "count": {"$count": { } }}
   }
]):
    pp.pprint(item)

{'_id': 'Headphones', 'count': 2}
{'_id': 'Phone', 'count': 3}
{'_id': 'Watch', 'count': 2}


4) Підрахуйте скільки є різних категорій товарів

In [146]:
len(items_collection.distinct("category"))

3

5) Виведіть список всіх виробників товарів без повторів

In [147]:
items_collection.distinct("producer")

['Apple', 'Samsung']

6) Напишіть запити, які вибирають товари за різними критеріям і їх сукупності:

a) категорія та ціна (в проміжку) - конструкція $and

In [148]:
for item in items_collection.find(
        {"$and": [
            {"category": "Phone"},
            {"price": {"$gt": 800, "$lt": 900}}
        ]}):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f11351'),
 'category': 'Phone',
 'display': 6.2,
 'model': 'Galaxy s23',
 'price': 850,
 'producer': 'Samsung'}


b) модель чи одна чи інша - конструкція $or

In [149]:
for item in items_collection.find(
        {"$or": [
            {"model": "iPhone 13"},
            {"model": "Galaxy Watch5"}
        ]}):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f11350'),
 'category': 'Phone',
 'display': 6.1,
 'model': 'iPhone 13',
 'price': 800,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 350,
 'producer': 'Samsung'}


c) виробники з переліку - конструкція $in

In [150]:
for item in items_collection.find(
        {
            "producer": {"$in": ["Samsung"]}
        }):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f11351'),
 'category': 'Phone',
 'display': 6.2,
 'model': 'Galaxy s23',
 'price': 850,
 'producer': 'Samsung'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 350,
 'producer': 'Samsung'}
{'Connectivity': 'Bluetooth 5.0',
 '_id': ObjectId('6429c7a7dd7672a8b4f11355'),
 'category': 'Headphones',
 'feature': 'IPX7 water-resistant',
 'model': 'Galaxy Buds2 Pro',
 'price': 125,
 'producer': 'Samsung'}


7) Оновить певні товари, змінивши існуючі значення і додайте нові властивості (характеристики) усім товарам за певним критерієм

In [151]:
items_collection.update_many(
    {"model": "iPhone 14"},
    {"$set": {"price": 1050}}
)
items_collection.update_many(
    { "price": {"$lt": 500}},
    {"$set": {"discount": True}}
)

In [152]:
for item in items_collection.find():
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f1134f'),
 'category': 'Phone',
 'display': 6.1,
 'model': 'iPhone 14',
 'price': 1050,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11350'),
 'category': 'Phone',
 'display': 6.1,
 'model': 'iPhone 13',
 'price': 800,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11351'),
 'category': 'Phone',
 'display': 6.2,
 'model': 'Galaxy s23',
 'price': 850,
 'producer': 'Samsung'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11352'),
 'category': 'Watch',
 'discount': True,
 'feature': 'ECG',
 'material': 'aluminium',
 'model': 'Series 8',
 'price': 400,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'discount': True,
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 350,
 'producer': 'Samsung'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11354'),
 'category': 'Headphones',
 'connectivity': 'Bluetooth 5.0',
 'discount': True,
 'feature': 'Force sensor',
 'model

8) Знайдіть товари у яких є (присутнє поле) певні властивості

In [153]:
for item in items_collection.find(
        { "material": { "$ne": None } }):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f11352'),
 'category': 'Watch',
 'discount': True,
 'feature': 'ECG',
 'material': 'aluminium',
 'model': 'Series 8',
 'price': 400,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'discount': True,
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 350,
 'producer': 'Samsung'}


9) Для знайдених товарів збільшіть їх вартість на певну суму

In [154]:
items_collection.update_many(
    {"material": { "$ne": None }},
    {"$inc": {"price": 10}}
)
for item in items_collection.find(
        { "material": { "$ne": None } }):
    pp.pprint(item)

{'_id': ObjectId('6429c7a7dd7672a8b4f11352'),
 'category': 'Watch',
 'discount': True,
 'feature': 'ECG',
 'material': 'aluminium',
 'model': 'Series 8',
 'price': 410,
 'producer': 'Apple'}
{'_id': ObjectId('6429c7a7dd7672a8b4f11353'),
 'category': 'Watch',
 'discount': True,
 'feature': 'Sleep Coaching',
 'material': 'aluminium',
 'model': 'Galaxy Watch5',
 'price': 360,
 'producer': 'Samsung'}


Товари ви додаєте в замовлення - orders, яке містити вартість, ім'я замовника, і адресу доставки.
Товари (items) в замовленні (order) повинні бути представлені у вигляді references, а замовник (customer) у вигляді embed


In [170]:
orders_collection = my_db["orders"]

1) Створіть кілька замовлень з різними наборами товарів, але так щоб один з товарів був у декількох замовленнях

In [186]:
orders_insert = [
    {
        "order_number": 1,
        "date": datetime.utcnow() - timedelta(days=13),
        "total_sum": 5432.1,
        "customer": {
            "name": "Vadym",
            "surname": "Popyk",
            "phones": [983456781, 988765432],
            "address": "Kyiv, UA"
        },
        "payment": {
            "card_owner": "Vadym Popyk",
            "cardId": 123654
        },
        "items_id": [ObjectId('6429c7a7dd7672a8b4f1134f'), ObjectId('6429c7a7dd7672a8b4f11352')]
    },
    {
        "order_number": 2,
        "date": datetime.utcnow() - timedelta(days=9),
        "total_sum": 891.4,
        "customer": {
            "name": "Vadym",
            "surname": "Popyk",
            "phones": [983456781, 988765432],
            "address": "Kyiv, UA"
        },
        "payment": {
            "card_owner": "Vadym Popyk",
            "cardId": 123654
        },
        "items_id": [ObjectId('6429c7a7dd7672a8b4f11354')]
    },
    {
        "order_number": 3,
        "date": datetime.utcnow() - timedelta(days=10),
        "total_sum": 1763.4,
        "customer": {
            "name": "Vasyl",
            "surname": "Hevko",
            "phones": [6522462, 5443534],
            "address": "Kyiv, UA"
        },
        "payment": {
            "card_owner": "Vasyl Hevko",
            "cardId": 75373
        },
        "items_id": [ObjectId('6429c7a7dd7672a8b4f1134f'), ObjectId('6429c7a7dd7672a8b4f11350'), ObjectId('6429c7a7dd7672a8b4f11351')]
    }
]

In [187]:
orders_collection.delete_many({})
orders_collection.insert_many(orders_insert)

2) Виведіть всі замовлення

In [184]:
for order in orders_collection.find({}):
    pp.pprint(order)

{'_id': ObjectId('6429c9b7dd7672a8b4f11359'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 30, 12, 48000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c9b7dd7672a8b4f1135a'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 24, 18, 30, 12, 48000),
 'items_id': [ObjectId('6429c7a7dd7672a8b4f1134f')],
 'order_number': 2,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 891.4}
{'_id': ObjectId('6429c9b7dd7672a8b4f1135b'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vasyl',
         

3) Виведіть замовлення з вартістю більше певного значення

In [159]:
for order in orders_collection.find({
    "total_sum": {"$gt": 5000}
}):
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}


4) Знайдіть замовлення зроблені одним замовником

In [160]:
for order in orders_collection.find({
    "customer.name": "Vadym",
    "customer.surname": "Popyk"
}):
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c81fdd7672a8b4f11357'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 24, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11346')],
 'order_number': 2,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 891.4}


5) Знайдіть всі замовлення з певним товаром (товарами) (шукати можна по ObjectId)


In [161]:
for order in orders_collection.find({
    "items_id": ObjectId('6429c5c2dd7672a8b4f11344')
}):
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c81fdd7672a8b4f11358'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vasyl',
              'phones': [6522462, 5443534],
              'surname': 'Hevko'},
 'date': datetime.datetime(2023, 3, 23, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11345'),
              ObjectId('6429c5c2dd7672a8b4f11346')],
 'order_number': 3,
 'payment': {'cardId': 75373, 'card_owner': 'Vasyl Hevko'},
 'total_sum': 1763.4}


6) Додайте в усі замовлення з певним товаром ще один товар і збільште існуючу вартість замовлення на деяке значення Х


In [162]:
orders_collection.update_many(
    {"items_id": ObjectId('6429c5c2dd7672a8b4f11346')},
    {"$push": {"items_id": ObjectId('6429c5c2dd7672a8b4f1134a')},
     "$inc": {"total_sum": 1000}}
)
for order in orders_collection.find():
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11344'),
              ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c81fdd7672a8b4f11357'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 24, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11346'),
              ObjectId('6429c5c2dd7672a8b4f1134a')],
 'order_number': 2,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 1891.4}
{'_id': ObjectId('6429c81fdd7672a8b4f11358'),
 'customer': {'address

7) Виведіть кількість товарів в певному замовленні

In [163]:
for order in orders_collection.aggregate(
    [
        {"$match" : {"_id": ObjectId('6429c81fdd7672a8b4f11358')} },
        {"$project": { "count": {"$size": "$items_id"}}},
    ]
):
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11358'), 'count': 4}


8) Виведіть тільки інформацію про кастомера і номери кредитної карт, для замовлень вартість яких перевищує певну суму

In [165]:
for order in orders_collection.find(
        {"total_sum": {"$gt": 5000}},
        {"customer": 1, "payment.cardId": 1}):
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'payment': {'cardId': 123654}}


9) Видаліть товар з замовлень, зроблених за певний період дат

In [166]:
orders_collection.update_many(
    {"date": {
        "$gt": datetime(2023, 3, 20),
        "$lt": datetime(2023, 3, 31)}
    },
    {"$pull": {"items_id": ObjectId('6429c5c2dd7672a8b4f11344')}}
)
for order in orders_collection.find():
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c81fdd7672a8b4f11357'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 24, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11346'),
              ObjectId('6429c5c2dd7672a8b4f1134a')],
 'order_number': 2,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 1891.4}
{'_id': ObjectId('6429c81fdd7672a8b4f11358'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vasyl',
      

10) Перейменуйте у всіх замовлення ім'я (прізвище) замовника

In [167]:
orders_collection.update_many(
    {},
    {"$set": {"customer.name": "Renamed"}}
)
for order in orders_collection.find():
    pp.pprint(order)

{'_id': ObjectId('6429c81fdd7672a8b4f11356'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Renamed',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 20, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11347')],
 'order_number': 1,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 5432.1}
{'_id': ObjectId('6429c81fdd7672a8b4f11357'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Renamed',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'date': datetime.datetime(2023, 3, 24, 18, 23, 26, 762000),
 'items_id': [ObjectId('6429c5c2dd7672a8b4f11346'),
              ObjectId('6429c5c2dd7672a8b4f1134a')],
 'order_number': 2,
 'payment': {'cardId': 123654, 'card_owner': 'Vadym Popyk'},
 'total_sum': 1891.4}
{'_id': ObjectId('6429c81fdd7672a8b4f11358'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Renamed',


11) Знайдіть замовлення зроблені одним замовником, і виведіть тільки інформацію про кастомера та товари у замовлені підставивши замість ObjectId("***") назви товарів та їх вартість (аналог join-а між таблицями orders та items).


In [188]:
for order in orders_collection.aggregate([
    {
        "$lookup": {
            "from": "items",
            "localField": "items_id",
            "foreignField": "_id",
            "as": "items"
        }
    },
    {
        "$addFields": {
            "items_price": "$items.price",
            "items_model": "$items.model"
        }
    },
    { "$project": {"customer": 1, "items_price": 1, "items_model": 1}},
]):
    pp.pprint(order)

{'_id': ObjectId('6429cc19dd7672a8b4f1135c'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'items_model': ['iPhone 14', 'Series 8'],
 'items_price': [1050, 410]}
{'_id': ObjectId('6429cc19dd7672a8b4f1135d'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vadym',
              'phones': [983456781, 988765432],
              'surname': 'Popyk'},
 'items_model': ['AirPods 3'],
 'items_price': [180]}
{'_id': ObjectId('6429cc19dd7672a8b4f1135e'),
 'customer': {'address': 'Kyiv, UA',
              'name': 'Vasyl',
              'phones': [6522462, 5443534],
              'surname': 'Hevko'},
 'items_model': ['iPhone 13', 'Galaxy s23', 'iPhone 14'],
 'items_price': [800, 850, 1050]}
